# Alaska Plotting Notebook
This notebook is a follow-on to the main Alaska notebook, taking the data generated and saved there to generate plots. Plots include surface height comparisons, snow depth comparisons, a field site map, and land cover boxplots.

Note that sometimes (read: frequently) Holoviews and Geoviews will not render the data in the notebook. However, saving the figure as an HTML and loading the file will render the plot correctly. It is not clear why this problem occurs.

In [ ]:
from bokeh.models import HoverTool
from cartopy import crs
import geopandas as gpd
import geoviews as gv
import geoviews.feature as gf
from geoviews import dim, opts
import geoviews.tile_sources as gts
gv.extension('bokeh')
import holoviews as hv
from holoviews import opts, Cycle
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point
import seaborn as sns
import scipy as sp
from scipy.stats import median_abs_deviation as mad
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter

In [ ]:
# Read in the data
dfsr = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_442010.csv')
df06 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06_snowdepth_rgt1356_cffl_2022321.csv')
df08 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl08_snowdepth_rgt1356_cffl_2022321.csv')

# Remove faulty values
dfsr['lidar_snow_depth'][dfsr['lidar_snow_depth']>2] = np.nan
df06['lidar_snow_depth'][df06['lidar_snow_depth']>2] = np.nan
dfsr['lidar_snow_depth'][dfsr['lidar_snow_depth']>2] = np.nan
df06['lidar_snow_depth'][df06['lidar_snow_depth']>2] = np.nan

dfsr['snow_depth_residual'][dfsr['snow_depth_residual'].abs()>2] = np.nan
df06['snow_depth_residual'][df06['snow_depth_residual'].abs()>2] = np.nan

## Along-track snow depths

Along-track line plot of snow depths derived by ICESat-2 and UAF. Only plots one IS-2 beam at a time.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12, 3))
#ATL06
ax[0].plot(df06['lat'][df06['beam']==5], df06['is2_snow_depth'][df06['beam']==5], label='ICESat-2')
ax[0].plot(df06['lat'][df06['beam']==5], df06['lidar_snow_depth'][df06['beam']==5], label='UAF')
ax[0].set_title('ATL06')
ax[0].set_ylabel('Snow depth [m]', fontsize=14)
ax[0].set_xlim([64.86, 64.888])
ax[0].set_ylim([0, 1.75])
ax[0].legend()
ax[0].yaxis.set_major_formatter('{x:9<3.1f}')
ax[0].set_xticks([64.86, 64.865, 64.87, 64.875, 64.88, 64.885])
ax[0].grid(True)

# ATL08
ax[1].plot(df08['lat'][df08['beam']==5], df08['is2_snow_depth'][df08['beam']==5], label='ICESat-2')
ax[1].plot(df08['lat'][df08['beam']==5], df08['lidar_snow_depth'][df08['beam']==5], label='UAF')
ax[1].set_title('ATL08')
ax[1].set_xlabel('Latitude', fontsize=14)
ax[1].set_xlim([64.86, 64.888])
ax[1].set_ylim([0, 1.75])
ax[1].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1].set_xticks([64.86, 64.865, 64.87, 64.875, 64.88, 64.885])
ax[1].grid(True)

# SlideRule
ax[2].plot(dfsr['lat'][(dfsr['beam']==50)],
           dfsr['is2_snow_depth'][(dfsr['beam']==50)],
           label='ICESat-2')
ax[2].plot(dfsr['lat'][(dfsr['beam']==50)],
           dfsr['lidar_snow_depth'][(dfsr['beam']==50)],
           label='UAF')
ax[2].set_title('SlideRule')
ax[2].set_xlim([64.86, 64.888])
ax[2].set_ylim([0, 1.75])
ax[2].yaxis.set_major_formatter('{x:9<3.1f}')
ax[2].set_xticks([64.86, 64.865, 64.87, 64.875, 64.88, 64.885])
fig.tight_layout()
ax[2].grid(True)

#plt.savefig('/home/jovyan/icesat2-snowex/figures/cffl/is2_uaf_line-plots_rgt1356_cffl_2022321.png', dpi=500)

## Snow depth scatter plot
Scatter plots of UAF vs. ICESat-2 snow depths. Seaborn is needed to construct the plots.

In [ ]:
# Derive r-squared values for each ICESat-2 product
df06_r2 = df06[df06['beam']==5].dropna()
linreg06 = sp.stats.linregress(df06_r2['lidar_snow_depth'], df06_r2['is2_snow_depth'])
df08_r2 = df08[df08['beam']==5].dropna()
linreg08 = sp.stats.linregress(df08_r2['lidar_snow_depth'], df08_r2['is2_snow_depth'])
dfsr_r2 = dfsr[(dfsr['beam']==50)].dropna()
linregsr = sp.stats.linregress(dfsr_r2['lidar_snow_depth'], dfsr_r2['is2_snow_depth'])

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,3))
# ATL06
sns.scatterplot(data=df06[df06['beam']==5], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[0])
ax[0].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[0].set_xlabel(' ')
ax[0].set_ylabel('ICESat-2 snow depth [m]', fontsize=14)
ax[0].set_xlim([0, 1.5])
ax[0].set_ylim([0, 1.5])
ax[0].xaxis.set_major_formatter('{x:9<3.1f}')
ax[0].yaxis.set_major_formatter('{x:9<3.1f}')
ax[0].annotate("r$^2$ = {:.3f}".format(linreg06.rvalue), (0.18, 0.9))
ax[0].grid(True, alpha=0.3)

# ATL08
sns.scatterplot(data=df08[df08['beam']==5], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[1])
ax[1].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[1].set_xlabel('UAF snow depth [m]', fontsize=14)
ax[1].set_ylabel(' ')
ax[1].set_xlim([0, 1.5])
ax[1].set_ylim([0, 1.5])
ax[1].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1].annotate("r$^2$ = {:.3f}".format(linreg08.rvalue), (0.18, 0.9))
ax[1].grid(True, alpha=0.3)

# SlideRule
sns.scatterplot(data=dfsr[(dfsr['beam']==50)], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[2])
ax[2].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[2].set_xlabel(' ')
ax[2].set_ylabel(' ')
ax[2].set_xlim([0, 1.5])
ax[2].set_ylim([0, 1.5])
ax[2].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2].yaxis.set_major_formatter('{x:9<3.1f}')
ax[2].annotate("r$^2$ = {:.3f}".format(linregsr.rvalue), (0.18, 0.9))
ax[2].grid(True, alpha=0.3)
plt.tight_layout()

#plt.savefig('/home/jovyan/icesat2-snowex/figures/cffl/is2_uaf_scatter-plots_rgt1356_cffl_2022321.png', dpi=500)

## Snow depth residual histograms
Plots normalized histograms of IS2-UAF depth residuals. The median bias and normalized median absolute deviation (NMAD) are also given.

In [ ]:
# Calculate weights for each histogram, to create normalized counts
w06 = np.ones_like(df06['snow_depth_residual'][df06['beam']==5])/float(len(df06['snow_depth_residual'][df06['beam']==5]))
w08 = np.ones_like(df08['snow_depth_residual'][df08['beam']==5])/float(len(df08['snow_depth_residual'][df08['beam']==5]))
wsr = np.ones_like(dfsr['snow_depth_residual'][(dfsr['beam']==50)])/float(len(dfsr['snow_depth_residual'][(dfsr['beam']==50)]))

# Calculate NMAD for each product
nmad06 = 1.4826*mad(df06['snow_depth_residual'][df06['beam']==5].dropna())
nmad08 = 1.4826*mad(df08['snow_depth_residual'][df08['beam']==5].dropna())
nmadsr = 1.4826*mad(dfsr['snow_depth_residual'][(dfsr['beam']==50)].dropna())

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,3))
# ATL06
sns.histplot(data=df06[df06['beam']==5], x='snow_depth_residual', ax=ax[0],
             bins=25,
             weights=w06)
ax[0].set_xlabel(' ')
ax[0].set_ylabel('Frequency [%]', fontsize=14)
ax[0].xaxis.set_major_formatter('{x:9<3.1f}')
ax[0].set_xlim([-1, 1])
ax[0].set_ylim([0, 0.25])
ax[0].yaxis.grid(True)
ax[0].annotate("Bias = {:.2f} cm".format(100*df06['snow_depth_residual'][df06['beam']==5].median()), (-0.85, 0.23))
ax[0].annotate("NMAD = {:.2f} cm".format(100*nmad06), (-0.85, 0.21))

# ATL08
sns.histplot(data=df08[df08['beam']==5], x='snow_depth_residual', ax=ax[1],
             bins=25,
             weights=w08)
ax[1].set_xlabel('IS2-UAF depth residual [m]', fontsize=14)
ax[1].set_ylabel(' ')
ax[1].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1].set_xlim([-1, 1])
ax[1].set_ylim([0, 0.25])
ax[1].yaxis.grid(True)
ax[1].annotate("Bias = {:.2f} cm".format(100*df08['snow_depth_residual'][df08['beam']==5].median()), (-0.85, 0.23))
ax[1].annotate("NMAD = {:.2f} cm".format(100*nmad08), (-0.85, 0.21))

# SlideRule
sns.histplot(data=dfsr[(dfsr['beam']==50)], x='snow_depth_residual', ax=ax[2],
             bins=25,
             weights=wsr)
ax[2].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2].set_xlabel(' ')
ax[2].set_ylabel(' ')
ax[2].set_xlim([-1, 1])
ax[2].set_ylim([0, 0.25])
ax[2].yaxis.grid(True)
ax[2].annotate("Bias = {:.2f} cm".format(100*dfsr['snow_depth_residual'][(dfsr['beam']==50)].median()), (-0.85, 0.23))
ax[2].annotate("NMAD = {:.2f} cm".format(100*nmadsr), (-0.85, 0.21))
plt.tight_layout()

#plt.savefig('/home/jovyan/icesat2-snowex/figures/cffl/is2_uaf_histogram-plots_rgt1356_cffl_2022321.png', dpi=500)

## Forest cover plots
Makes two plots related to forest cover: (1) a histogram of forest covers, rounded to the nearest multiple of 10; (2) boxplot of depth residuals, distributed by forest cover.

In [ ]:
# Round forest cover percentages to nearest multiple of 10
dfsr['forest_cover'] = round(dfsr['forest_cover']/10)*10

# Generate weights for forest cover histogram
weights = np.ones_like(np.array(dfsr[dfsr['beam']==50]['forest_cover']))/float(len(np.array(dfsr[dfsr['beam']==50]['forest_cover'])))

# Create a (temporary) new dataframe that groups depth residuals by forest cover
new_df = dfsr[dfsr['beam']==50].reset_index().melt(id_vars=['forest_cover'], value_vars=['snow_depth_residual'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))
# Histogram plot
sns.histplot(data=dfsr[dfsr['beam']==50], x='forest_cover', ax=ax[0],
             bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
             weights=weights)
ax[0].set_xlim([0, 80])
ax[0].set_xlabel('Forest cover [%]', fontsize=14)
ax[0].set_ylabel('Frequency [%]', fontsize=14)
ax[0].set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80])
ax[0].yaxis.grid(True)

# Boxplot
sns.boxplot(data=new_df, x='forest_cover', y='value', ax=ax[1], color='C0')
ax[1].set_title(' ')
ax[1].set_xlabel('Forest cover [%]', fontsize=14)
ax[1].set_ylabel('IS2-UAF depth residual [m]', fontsize=14)
ax[1].grid(True)
#ax[1].set_xticklabels([0, 10, 20, 30, 40, 50, 60, 70])
#ax[1].set_ylim([-1, 0.5])
#ax[1].set_xlim([0.5, 9.5])
fig.tight_layout()
plt.suptitle("Creamer's Field", fontsize=14)
#plt.savefig('/home/jovyan/icesat2-snowex/figures/cffl/is2_uaf_forest-cover-plots_rgt1356_cffl_2022321.png', dpi=500)

## Photon number plots
Same as the forest cover figure, but instead using the number of photons used in each SlideRule segment.

In [ ]:
# Round photon numbers to nearest multiple of 20
dfsr['n_fit_photons'] = round(dfsr['n_fit_photons']/20)*20

# Generate weights for histogram plot
weights = np.ones_like(np.array(dfsr[dfsr['beam']==50]['n_fit_photons']))/float(len(np.array(dfsr[dfsr['beam']==50]['n_fit_photons'])))

# Create new dataframe that groups residuals by photon count
new_df = dfsr[dfsr['beam']==50].reset_index().melt(id_vars=['n_fit_photons'], value_vars=['snow_depth_residual'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))
# Histogram plot
sns.histplot(data=dfsr[dfsr['beam']==50], x='n_fit_photons', ax=ax[0],
             bins=[20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
             weights=weights)
#ax[0].set_xlim([0, 80])
ax[0].set_ylim([0, 0.35])
ax[0].set_xlabel('Number of fitted photons', fontsize=14)
ax[0].set_ylabel('Frequency [%]', fontsize=14)
ax[0].set_xticks([20, 40, 60, 80, 100, 120, 140, 160, 180, 200])
ax[0].yaxis.grid(True)

# Boxplot
sns.boxplot(data=new_df, x='n_fit_photons', y='value', ax=ax[1], color='C0')
ax[1].set_title(' ')
ax[1].set_ylim([-0.7, 0.7])
ax[1].set_xlabel('Number of fitted photons', fontsize=14)
ax[1].set_ylabel('IS2-UAF depth residual [m]', fontsize=14)
ax[1].grid(True)
ax[1].set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
ax[1].set_xticklabels([20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0])
#ax[1].set_ylim([-1, 0.5])
ax[1].set_xlim([-0.5, 10.5])
fig.tight_layout()
#plt.suptitle("Creamer's Field", fontsize=14)
#plt.savefig('/home/jovyan/icesat2-snowex/figures/bcef/is2_uaf_number-photon-plots_rgt472_bcef_2022423.png', dpi=500)

## SlideRule segment plots
Figures using SlideRule data of differing resolution, segment length, etc. Currently a work in progress.

In [ ]:
dfsr_142010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/acp/atl06sr_snowdepth_rgt1097_acp_202234_142010.csv')
dfsr_4410050 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/acp/atl06sr_snowdepth_rgt1097_acp_202234_4410050.csv')
dfsr_44105 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/acp/atl06sr_snowdepth_rgt1097_acp_202234_44105.csv')
dfsr_444020 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/acp/atl06sr_snowdepth_rgt1097_acp_202234_444020.csv')

dfsr_142010['is2_snow_off'] = dfsr_142010['is2_height'] - dfsr_142010['lidar_snow_depth']
dfsr_4410050['is2_snow_off'] = dfsr_4410050['is2_height'] - dfsr_4410050['lidar_snow_depth']
dfsr_44105['is2_snow_off'] = dfsr_44105['is2_height'] - dfsr_44105['lidar_snow_depth']
dfsr_444020['is2_snow_off'] = dfsr_444020['is2_height'] - dfsr_444020['lidar_snow_depth']